<a href="https://colab.research.google.com/github/windaapriliyanti/Myskill/blob/main/TensorFlow_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Datasets

TFDS menyediakan kumpulan dataset siap pakai untuk digunakan dengan TensorFlow, Jax, dan kerangka pembelajaran mesin lainnya.

Ini menangani pengunduhan dan persiapan data secara deterministik dan membangun `tf.data.Dataset` (atau `np.array`).

Catatan: Jangan bingung antara [TFDS](https://www.tensorflow.org/datasets) (perpustakaan ini) dengan `tf.data` (API TensorFlow untuk membangun pipa data efisien). TFDS adalah lapisan tinggi yang melibatkan `tf.data`. Jika Anda tidak familiar dengan API ini, kami sarankan membaca [panduan resmi tf.data](https://www.tensorflow.org/guide/data) terlebih dahulu.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Dataset yang tersedia di Tensorflow

Semua builder dataset adalah turunan dari `tfds.core.DatasetBuilder`. Untuk mendapatkan daftar pembangun yang tersedia, gunakan `tfds.list_builders()` atau lihat [katalog dataset](https://www.tensorflow.org/datasets/catalog/overview).

In [ ]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asqa',
 'asset',
 'assin2',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'ble_wind_field',
 'blimp',
 'booksum',
 'bool_q',
 'bot_adversarial_dialogue',
 'bucc',
 'c4',
 'c4_wsrs',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cardiotox',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'cherry_blossoms',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar100_n',
 'cifar10_1',
 'cifar10_corrupted',
 'cifar10_h',
 'cifar10_n',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'conll2002',
 

## Bangun end to end pipeline

Demonstrasi cara pembuatan tensorflow dataset untuk training model.


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

### Step 1: Bangun input pipeline

referensi membangun input pipeline yang efisien:
* The [Performance tips](https://www.tensorflow.org/datasets/performances) guide
* The [Better performance with the `tf.data` API](https://www.tensorflow.org/guide/data_performance#optimize_performance) guide

#### Load a dataset

Kita akan menggunakan data MNIST:

* `shuffle_files=True`:Data MNIST hanya disimpan dalam satu file, tetapi untuk dataset yang lebih besar dengan beberapa file di disk, praktik yang baik adalah untuk mengacaknya saat pelatihan.
* `as_supervised=True`: Returns tuple `(img, label)` bukan dictionary `{'image': img, 'label': label}`.

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.




Terapkan transformasi berikut:

* `tf.data.Dataset.map`: TFDS menyediakan gambar dengan tipe `tf.uint8`, sementara model mengharapkan tipe `tf.float32`. Oleh karena itu, Anda perlu normalisasi gambar.
* `tf.data.Dataset.cache`: Saat Anda memuat dataset ke dalam memori, simpan di cache sebelum diacak untuk kinerja yang lebih baik.<br/>
__Catatan:__ Transformasi acak harus diterapkan setelah penyimpanan di cache.
* `tf.data.Dataset.shuffle`: Untuk keacakan sejati, atur buffer pengacakan sebesar ukuran dataset penuh.<br/>
__Catatan:__ Untuk dataset besar yang tidak muat dalam memori, gunakan `buffer_size=1000` jika sistem Anda mendukungnya.
* `tf.data.Dataset.batch`: Gabungkan elemen dataset setelah diacak untuk mendapatkan batch unik pada setiap epoch.
* `tf.data.Dataset.prefetch`: Praktik yang baik untuk mengakhiri pipa data dengan prefetching [untuk kinerja](https://www.tensorflow.org/guide/data_performance#prefetching).

## Training Set

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

## Evaluation Set

Pipeline data pengujian Anda mirip dengan pipeline data pelatihan dengan perbedaan kecil:

* Anda tidak perlu memanggil `tf.data.Dataset.shuffle`.
* Penyimpanan di cache dilakukan setelah pengelompokan karena batch dapat sama antara epoch.

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

### Step 2: Build and Train Model
Gunakan train set dan test set untuk melatih modelnya

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),  # Increased the number of neurons for better representation
    tf.keras.layers.Dropout(0.5),  # Added dropout for regularization
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Added dropout for regularization
    tf.keras.layers.Dense(10)
])

# Compile the optimized model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Explicitly set learning rate
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 15s 10ms/step - loss: 0.5259 - sparse_categorical_accuracy: 0.8371 - val_loss: 0.1687 - val_sparse_categorical_accuracy: 0.9489
Epoch 2/6
469/469 [==============================] - 2s 5ms/step - loss: 0.2458 - sparse_categorical_accuracy: 0.9287 - val_loss: 0.1291 - val_sparse_categorical_accuracy: 0.9612
Epoch 3/6
469/469 [==============================] - 2s 5ms/step - loss: 0.1927 - sparse_categorical_accuracy: 0.9438 - val_loss: 0.1059 - val_sparse_categorical_accuracy: 0.9684
Epoch 4/6
469/469 [==============================] - 3s 5ms/step - loss: 0.1663 - sparse_categorical_accuracy: 0.9520 - val_loss: 0.0966 - val_sparse_categorical_accuracy: 0.9703
Epoch 5/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1437 - sparse_categorical_accuracy: 0.9580 - val_loss: 0.0914 - val_sparse_categorical_accuracy: 0.9730
Epoch 6/6
469/469 [==============================] - 2s 5ms/step - loss: 0.1314 - sparse_categorical_ac